In [169]:
import tensorflow as tf
import numpy as np
import pandas
from pandas import DataFrame
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
from sklearn import svm, preprocessing


In [170]:
class Indicator:

    def __init__(self):

        try:
            con = sqlite3.connect("new_kospi.db")
            cursor = con.cursor()
            cursor.execute("SELECT * FROM new_kospi")
            rows = cursor.fetchall()

            self.k_date,self.jisu, self.highjisu, self.lowjisu = [],[],[],[]

            for row in rows:
                self.k_date.append(int(row[0]))
                self.jisu.append(float(row[1]))
                self.highjisu.append(float(row[2]))
                self.lowjisu.append(float(row[3]))


            
            self.k_date.reverse()
            self.jisu.reverse()
            self.highjisu.reverse()
            self.lowjisu.reverse()

            self.arr_date= np.array(self.k_date)
            self.arr_close= np.array(self.jisu)
            self.arr_high= np.array(self.highjisu)
            self.arr_low= np.array(self.lowjisu)
        except:
            print("error database connection")


In [171]:
FEATURES = ['high','low','close']

A = Indicator()
data = {'year': A.arr_date,
        'close': A.arr_close,
        'high': A.arr_high,
        'low': A.arr_low}
frame = DataFrame(data, columns= ['year','close', 'high','low'])

In [189]:
new_XX = np.array(frame[FEATURES])[:-1,]
x_data = preprocessing.scale(new_XX)
y_data = np.roll(frame['close'][:].tolist(),-1)[:-1]

In [192]:
x_data.shape

(7216, 3)

In [193]:
y_data.shape

(7216,)

In [194]:
y_data = y_data.reshape(7216,1)

In [195]:
# Parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 9 # 1st layer number of features
n_input = 3 # MNIST data input (img shape: 28*28)
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [196]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [197]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.square(pred - y)/2
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [200]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={x:x_data, y:y_data})
        if step%20 ==0:
#             print (step, sess.run(cost, feed_dict={x:x_data, y:y_data}))
            print (step, sess.run(cost, feed_dict={x:x_data, y:y_data}), sess.run(weights), sess.run(biases))

0 [[  391139.46875]
 [  393735.4375 ]
 [  395886.1875 ]
 ..., 
 [ 1962934.625  ]
 [ 1961563.625  ]
 [ 1966234.5    ]] {'out': array([[ 0.34942868],
       [ 0.0109567 ],
       [ 0.84756583],
       [ 0.18337516],
       [-2.50786209],
       [ 0.66233176],
       [ 0.37877458],
       [-0.32512537],
       [ 0.54372162]], dtype=float32), 'h1': array([[ 1.59621215, -1.06542265, -0.54848367,  1.380054  , -1.37931967,
         1.79344714, -1.18033981, -1.37473154, -1.74605489],
       [-0.03507555, -0.36301747, -0.87108999, -0.2098802 ,  0.34255728,
         0.85337579,  1.61503804,  1.43995595, -0.68141741],
       [ 0.11424308,  0.61817139,  1.28751087, -0.08777413,  0.24485235,
        -0.57424259, -0.07571107,  3.0821116 , -0.9316656 ]], dtype=float32)} {'out': array([-0.38852799], dtype=float32), 'b1': array([ 0.17549844,  0.00338206,  0.09846481,  0.74840909,  0.87075347,
        0.21615621, -0.31528693, -0.61510545, -1.16324687], dtype=float32)}
20 [[  388972.875  ]
 [  391584.25 

In [ ]:

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})